In [352]:
from simplech import ClickHouse
import ujson
import requests
import datetime, time

In [253]:
ch = ClickHouse()

In [361]:
# Authentification
client_id = '82c7d4cc-5ghhd-6dhd-9873-06a22f2117d2'
client_secret = 'd061JHDghshsqKuFE+iSqcKdhjsjhhwCXNh82m2LY='
username =  'test@test.ru'
password = 'Daksjhdj4'

# Global for interact with powerbi
tableName = 'Data'
dataset_id = ''
token ={}

In [397]:
# Global for CreateDataset in power Bi 
datasetName = 'From CLickhouse 3'

In [406]:
# Query for Clickhouse without FORMAT and LIMIT
# formatDateTime(dateTime,'%FT%T') - format for powerbi 
# Recreate dataset or Update if query is changed 
query="""
    SELECT data_dealerId, data_category, data_action, formatDateTime(dateTime,'%FT%T') AS dateTime, uniq(uid) AS users
    FROM gaevents
    WHERE data_dealerId !=''
    GROUP BY data_category, data_action, timestamp, data_dealerId, dateTime
    ORDER BY users DESC
"""

### Create dataset schema in power bi from query 
    push_dataset_schema(datasetName,tableName,query)

### Push data to power bi
 default limit is 10000 rows 
    
    push_data_to_datatset(dataset_id,tableName,query,limit)
### Cheak dataset Schema from query 
    create_dataset_json(datasetName, tableName, json_Schema_From_Query(query))

In [408]:
def json_Pbi_From_Clichouse(query, limit=None):
    '''
    Получаем значения из Clickhouse в формате json и упаковываем в json строку валидную для power bi 
    '''
    if limit:  query += f' LIMIT {limit}'
    query += ' FORMAT JSON'
    result = ch.select(query)
    data_string = ujson.dumps(ujson.loads(result).get('data'))
    json_string='{"rows":'+data_string+'}'
    return json_string

In [392]:
def json_Schema_From_Query(query, limit=1, custom_datetime_column=''):
    '''
    Формируем таблицу соответствия типов данных Clichouse -> PowerBi
    custom_datetime_column - название для поля даты/вермя если оно отличается от стандартных
    '''
    if limit:  query += f' LIMIT {limit}'
    query += ' FORMAT JSON'
    result = ch.select(query)
    new_list = []
    for row in ujson.loads(result).get('meta'):
        new_dict = {}
        new_dict['name'] = row ['name']
        if row['name'].lower() in ('datetime', custom_datetime_column.lower()):
            new_dict['dataType'] = 'DateTime'
        else:
            new_dict['dataType'] = pbiDataType(row['type'])
        new_list.append(new_dict)
    return new_list

In [404]:
def connect_pbi(username,password,client_id,client_secret):
    url = 'https://login.microsoftonline.com/common/oauth2/token'
    data = {
            'grant_type': 'password',
            'scope': 'https://api.powerbi.com',
            'resource': 'https://analysis.windows.net/powerbi/api',
            'client_id': client_id,
            'client_secret':client_secret,
            'username': username,
            'password': password
           }
    r = requests.post(url, data=data)
    if r.status_code == HTTP_OK:
        global token
        token['access_token'] = r.json().get('access_token')
        token['expires_token'] = r.json().get('expires_on')
        return True 
    else:
        print('Ошибка подключения')
        return False

In [346]:
def verify_token():
    '''
    Проверяет истек ли токен, если да то обновляет его
    '''
    global token
    expires_token = datetime.datetime.fromtimestamp(int(token['expires_token']))
    if token['access_token'] == None:
        print('Токен не был получен')
        return False
    else:
        if expires_token < datetime.datetime.now():
            connect_pbi(username,password,client_id,client_secret)
            return True
        else:
            return True

In [355]:
def get_headers():
    """Create HTTP headers"""
    if verify_token() == True:
        headers = {
            #"Content-Type": "application/json; charset=utf-8",
            "Authorization": f"Bearer {token['access_token']}"
        }
        return headers
    else:
        return print('Проблема с токеном')


In [60]:
def pbiDataType(type):
    type = type.lower()
    if type in ("uint16", "int16","int8", "uint8"):
      return "Int16"
    elif type in ("uint32", "int32"):
      return "Int32"
    elif type in ("uint64", "int64"):
      return "Int64"
    elif type in ("datetime","date"):
      return "DateTime"
    elif type == "float32":
      return "Float"
    elif type == "string":
      return "String"
    return None

In [399]:
def create_dataset_json(datasetName,tableName,schema):
    json_schema_string = ujson.dumps(schema)
    body = f'{{"name":"{datasetName}","defaultMode":"Push","tables":['
    body += f'{{"name":"{tableName}","columns":{json_schema_string}}}'
    body += ']}'
    return body

In [400]:
def push_dataset_schema(datasetName,tableName, query):
    body = create_dataset_json(datasetName,tableName,json_Schema_From_Query(query)) 
    url = 'https://api.powerbi.com/v1.0/myorg/datasets'
    r = requests.post(url=url,data=body, headers=getHeaders())
    global dataset_id
    dataset_id = r.json().get('id')
    return dataset_id

In [407]:
def push_data_to_powerbi(dataset_id,tableName,query,limit=10000):
    url = f'https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/tables/{tableName}/rows'
    data_json = json_Pbi_From_Clichouse(query, limit)
    r = requests.post(url=url, data=data_json, headers=getHeaders())
    if r.status_code == HTTP_OK:
        return 'Успешно импортировано'
    else:
        return f'Ошибка {r.status_code} {r.json()}'